In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load dataset
train_file_path = '/kaggle/input/quora-question-pairs/train.csv.zip'
test_file_path = '/kaggle/input/quora-question-pairs/test.csv'

# Load CSV files
train_df = pd.read_csv(train_file_path, compression='zip')
test_df = pd.read_csv(test_file_path)

# Split training data
df_train, df_val = train_test_split(train_df, test_size=0.1, random_state=42)

# Define BiEncoder with Cosine Similarity
class BiEncoderCosineSimilarity(torch.nn.Module):
    def __init__(self, model_name):
        super(BiEncoderCosineSimilarity, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        outputs1 = self.model(input_ids1, attention_mask=attention_mask1)
        outputs2 = self.model(input_ids2, attention_mask=attention_mask2)
        embeddings1 = outputs1.last_hidden_state[:, 0, :]
        embeddings2 = outputs2.last_hidden_state[:, 0, :]
        similarity = F.cosine_similarity(embeddings1, embeddings2)
        return similarity

# Function to create DataLoader
def create_data_loader(df, tokenizer, max_len, batch_size):
    question1_list = df['question1'].astype(str).tolist()
    question2_list = df['question2'].astype(str).tolist()
    
    # Tokenize the inputs
    encodings = tokenizer(
        text=question1_list,
        text_pair=question2_list,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    input_ids1 = encodings['input_ids']
    attention_mask1 = encodings['attention_mask']
    input_ids2 = encodings['token_type_ids']  # Use token_type_ids as input_ids2
    attention_mask2 = encodings['attention_mask']
    
    labels = torch.tensor(df['is_duplicate'].values)
    
    dataset = TensorDataset(input_ids1, attention_mask1, input_ids2, attention_mask2, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model and training setup
model_name = 'bert-base-uncased'
bi_encoder_cosine = BiEncoderCosineSimilarity(model_name)
tokenizer = bi_encoder_cosine.tokenizer
max_len = 128
batch_size = 16

train_loader = create_data_loader(df_train, tokenizer, max_len, batch_size)
val_loader = create_data_loader(df_val, tokenizer, max_len, batch_size)

optimizer = torch.optim.Adam(bi_encoder_cosine.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bi_encoder_cosine.to(device)

for epoch in range(2):  # Example: 3 epochs
    bi_encoder_cosine.train()
    running_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
        input_ids1, attention_mask1, input_ids2, attention_mask2, labels = batch
        input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
        input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)
        labels = labels.to(device).float()

        optimizer.zero_grad()
        outputs = bi_encoder_cosine(input_ids1, attention_mask1, input_ids2, attention_mask2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader)}")

    # Validation loop
    bi_encoder_cosine.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids1, attention_mask1, input_ids2, attention_mask2, labels = batch
            input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
            input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)
            labels = labels.to(device).float()

            outputs = bi_encoder_cosine(input_ids1, attention_mask1, input_ids2, attention_mask2)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    print(f"Validation Loss: {val_loss / len(val_loader)}")

# Save the model
torch.save(bi_encoder_cosine.state_dict(), '/kaggle/working/bi_encoder_cosine.pth')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Validation Loss: 0.4328297205390221


Epoch 2: 100%|██████████| 22742/22742 [4:19:17<00:00,  1.46it/s]


Epoch 2 Loss: 0.41333272489936546


Validation: 100%|██████████| 2527/2527 [10:29<00:00,  4.02it/s]


Validation Loss: 0.4222930433441633
